In [49]:
import random
from math import ceil
from typing import Dict, Tuple
from typing import List, Set

import torch
from torch import Tensor
from torch import nn
from torch.utils.data import Dataset, DataLoader

from graph import Graph
from part import Part
from preprocess import create_family_id_mapping
from utils import load_graphs
from matplotlib import pyplot as plt
from tqdm import tqdm

In [33]:
graphs = load_graphs()
mapping = create_family_id_mapping(graphs)

seed = 7
random = random.Random(seed)
random.shuffle(graphs)

training_set_size_ratio = 0.8
validation_set_size_ratio = 0.1
batch_size = 1024

input_size = 2 * len(mapping)  # 184
output_size = len(mapping)  # 92

input_size = input_size
hidden_size = 2 * input_size
output_size = output_size

learning_rate = 0.0005
num_epochs = 50

In [34]:
def create_features(graph: Graph, dense_family_id_mapping: Dict) -> Tuple[List[Tensor], List[Tensor]]:
    num_different_family_ids = len(dense_family_id_mapping)

    # Create a tensor for existing nodes and count their occurrences
    all_nodes_tensor = torch.zeros(num_different_family_ids, dtype=torch.float)
    nodes = graph.get_nodes()
    edges = graph.get_edges()

    feature_tensors = []
    target_tensors = []

    for node in nodes:
        dense_family_id = dense_family_id_mapping[int(node.get_part().get_family_id())]
        all_nodes_tensor[dense_family_id] += 1.0

        target_tensor = torch.zeros(num_different_family_ids, dtype=torch.float)
        for neighbour_node in edges[node]:
            neighbour_node_dense_family_id = dense_family_id_mapping[int(neighbour_node.get_part().get_family_id())]
            target_tensor[neighbour_node_dense_family_id] += 1
        target_tensors.append(target_tensor)

    for node in nodes:
        dense_family_id = dense_family_id_mapping[int(node.get_part().get_family_id())]
        given_node_tensor = torch.zeros(num_different_family_ids, dtype=torch.float)
        given_node_tensor[dense_family_id] = 1

        feature_tensors.append(torch.cat((all_nodes_tensor, given_node_tensor), dim=-1))

    return feature_tensors, target_tensors

In [35]:
# split data into train, val and test - they are already shuffled

train_upper = ceil(training_set_size_ratio * len(graphs))
val_upper = ceil((training_set_size_ratio + validation_set_size_ratio) * len(graphs))

train_graphs = graphs[0:train_upper]
val_graphs = graphs[train_upper:val_upper]
test_graphs = graphs[val_upper:len(graphs) + 1]


class GraphDataset(Dataset):
    def __init__(self, graphs: List[Graph]):
        x_list = []
        y_list = []
        for graph in graphs:
            x_per_graph, y_per_graph = create_features(graph, mapping)
            x_list += x_per_graph
            y_list += y_per_graph
        self.x_tensor = torch.stack(x_list)
        self.y_tensor = torch.stack(y_list)
        self.n_samples = self.y_tensor.shape[0]

    def __getitem__(self, index):
        return self.x_tensor[index], self.y_tensor[index]

    def __len__(self):
        return self.n_samples


train_loader = DataLoader(
    dataset=GraphDataset(train_graphs),
    batch_size=batch_size,
    shuffle=False,
)
val_loader = DataLoader(
    dataset=GraphDataset(val_graphs),
    batch_size=batch_size,
    shuffle=False,
)

In [57]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        # self.fc2 = nn.Linear(..., ...)
        self.fc3 = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        # x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x


device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.has_mps else "cpu")
model = Net().to(device)
n_total_steps = len(train_loader)

optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
loss_fn = nn.BCELoss()
train_losses = []
val_losses = []

train_accuracies = []
val_accuracies = []

for epoch in tqdm(range(num_epochs),ascii="░▒█"):
    train_losses_per_epoch = 0.0
    train_accuracy_per_epoch = 0.0
    for i, (x_train, y_train) in enumerate(train_loader):
        x_train = x_train.to(device)
        y_train = y_train.to(device)
        y_pred = model(x_train)
        loss = loss_fn(y_pred, y_train)
        train_losses_per_epoch += loss.cpu().detach().numpy()
        train_accuracy = torch.sum(y_train == y_pred.round()) / torch.numel(y_train)
        train_accuracy_per_epoch += train_accuracy.cpu().detach().numpy()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    train_losses.append(train_losses_per_epoch / len(train_loader))
    train_accuracies.append(train_accuracy_per_epoch / len(train_loader))

    with torch.no_grad():
        val_losses_per_epoch = 0.0
        val_accuracies_per_epoch = 0.0
        for i, (x_val, y_val) in enumerate(val_loader):
            x_val = x_val.to(device)
            y_val = y_val.to(device)
            y_pred = model(x_val)
            loss = loss_fn(y_pred, y_val)
            val_accuracy = torch.sum(y_val == y_pred.round()) / torch.numel(y_val)
            val_losses_per_epoch += loss.cpu()
            val_accuracies_per_epoch += val_accuracy.cpu()
        val_losses.append(val_losses_per_epoch / len(val_loader))
        val_accuracies.append(val_accuracies_per_epoch / len(val_loader))


100%|██████████| 50/50 [00:31<00:00,  1.61it/s]


In [ ]:
plt.plot(train_losses, label="Training Loss")
plt.plot(val_losses, label="Validation Loss")
plt.legend()
# plt.yscale('log')
plt.show()

plt.plot(train_accuracies, label="Train Accuracy")
plt.plot(val_accuracies, label="Validation Accuracy")
plt.legend()
# plt.yscale('log')
plt.show()

In [54]:
from evaluation import edge_accuracy

dense_family_id_to_family_id = {v: k for k, v in mapping.items()}
edge_accuracies = []
cpu_model = model.cpu()
with torch.no_grad():
    with tqdm(val_graphs) as val_graphs_with_progress:
        for index, test_graph in enumerate(val_graphs_with_progress):
            # matrix dimensions: x = nodes_list index, y = dense family id
            predicted_adjacency_matrix = []
            predicted_graph = Graph()
            x, y = create_features(test_graph, mapping)
            for x_test, y_test in zip(x, y):
                y_test_pred = cpu_model(x_test)
                predicted_adjacency_matrix.append(y_test_pred)
            predicted_adjacency_matrix = torch.stack(predicted_adjacency_matrix)

            nodes = test_graph.get_nodes()
            nodes_list = list(nodes)
            node_count = len(nodes)
            added_nodes = set()
            last_rep_node_a_index = -1
            last_node_b_dense_family_id = -1
            predicted_graph_edge_count = 0

            while True:
                max_pos_tensor = (predicted_adjacency_matrix == torch.max(predicted_adjacency_matrix)).nonzero()
                rep_node_a_index = max_pos_tensor[0][0].item()
                node_b_dense_family_id = max_pos_tensor[0][1].item()

                if rep_node_a_index == last_rep_node_a_index and node_b_dense_family_id == last_node_b_dense_family_id:
                    break
                last_rep_node_a_index = rep_node_a_index
                last_node_b_dense_family_id = node_b_dense_family_id

                node_a_family_id = int(nodes_list[rep_node_a_index].get_part().get_family_id())
                node_b_family_id = dense_family_id_to_family_id[node_b_dense_family_id]
                node_a_dense_family_id = mapping[node_a_family_id]

                for node_a_index, node_a in enumerate(nodes_list):
                    if int(node_a.get_part().get_family_id()) == node_a_family_id:
                        for node_b_index, node_b in enumerate(nodes_list):
                            if int(node_b.get_part().get_family_id()) == node_b_family_id:
                                if not (node_a in added_nodes and node_b in added_nodes):
                                    predicted_graph.add_undirected_edge(node_a.get_part(), node_b.get_part())
                                    predicted_graph_edge_count += 1
                                    added_nodes.add(node_a)
                                    added_nodes.add(node_b)

                                # non-necessary performance optimization
                                predicted_adjacency_matrix[node_a_index][node_b_dense_family_id] = 0.0
                                predicted_adjacency_matrix[node_b_index][node_a_dense_family_id] = 0.0
                predicted_adjacency_matrix[rep_node_a_index][node_b_dense_family_id] = 0.0
                if predicted_graph_edge_count == node_count - 1:
                    break
            graph_edge_accuracy = edge_accuracy(test_graph, predicted_graph) / (node_count * node_count)
            # print(f"edge_accuracy: {graph_edge_accuracy}")
            # test_graph.draw()
            # predicted_graph.draw()
            # print("----")

            edge_accuracies.append(graph_edge_accuracy)
sum(edge_accuracies) / len(edge_accuracies)

  2%|▏         | 23/1116 [00:02<01:40, 10.86it/s]


KeyboardInterrupt: 

In [39]:
# Test accuracy of randomly generated trees
edge_accuracies = []
for test_graph in test_graphs[:100]:
    predicted_graph = Graph()
    nodes = test_graph.get_nodes()
    node_count = len(nodes)
    random_nodes_list = list(nodes)
    random.shuffle(random_nodes_list)
    node_a_index = 0
    node_b_index = 1

    for node_a_index in range(node_count - 1):
        existing_node_index = random.randint(0, node_a_index)
        existing_node = random_nodes_list[existing_node_index]
        new_node_index = node_a_index + 1
        new_node = random_nodes_list[new_node_index]
        predicted_graph.add_undirected_edge(existing_node.get_part(), new_node.get_part())

    edge_accuracies.append(edge_accuracy(test_graph, predicted_graph) / (node_count * node_count))

sum(edge_accuracies) / len(edge_accuracies)

0.6657158222270694